In [ ]:
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.pipeline import Pipeline

import boto3
from sagemaker.workflow.function_step import step
from sagemaker import get_execution_role, Session
import logging

In [ ]:
from fetch import fetch
from preprocess import preprocess

In [ ]:
sagemaker_session = Session()
role = get_execution_role()

default_bucket = sagemaker_session.default_bucket()
session = PipelineSession(boto_session=sagemaker_session.boto_session, default_bucket=default_bucket)
pipeline_name = 'stock-pipeline'
default_ticker = "SPY"
years_of_data_to_fetch = 1

In [ ]:
@step(
    instance_type="ml.m5.large"
)
def train(data):
    # retrieve preprocessed csv data
    # train model
    # save model to s3 bucket /model with postfix of date and RMSE in filename
    logging.info(f'data should be output of preprocess function: {data}')
    return "stub_train"

In [ ]:
@step(
    instance_type="ml.m5.large"
)
def eval(data):
    # load current model file and latest
    # extract model performance from RSME in filename
    # if model is better than current
    # pass True to deploy phase (Use conditional to assess when to run deploy)
    logging.info(f'data should be output of preprocess function: {data}')
    return False

In [ ]:
@step(
    instance_type="ml.m5.large"
)
def deploy(data):
    # if latest_model is different than current
    # update deployment to use latest model
    logging.info(f'data should be output of train function: {data}')
    return "stub_eval"

In [ ]:
fetch_result = fetch(default_ticker, years_of_data_to_fetch)
preprocess_result = preprocess(fetch_result)
train_result = train(preprocess_result)
eval_result = eval(train_result)
deploy_result = deploy(eval_result)

pipeline = Pipeline(
    name=pipeline_name,
    steps=[fetch_result, preprocess_result, train_result, eval_result, deploy_result],
    sagemaker_session=session,
)

In [ ]:
pipeline.upsert(role)